In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [3]:
REBUILD_DATA = True # set to true to one once, then back to false unless you want to change something in your training data.

class DogsVSCats():
    IMG_SIZE = 50
    CATS = "kagglecatsanddogs_3367a/PetImages/Cat"
    DOGS = "kagglecatsanddogs_3367a/PetImages/Dog"
    #TESTING = "PetImages/Testing"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []

    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)
                    #color data is not needed
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    #resizing or normalizing data to type 50*50 pixels
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    #making training data = image array + one hot encoding for dog or cat
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  #t 
                    #print(np.eye(2)[self.LABELS[label]])

                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1

                except Exception as e:
                    pass
                    #print(label, f, str(e))

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Cats:',self.catcount)
        print('Dogs:',self.dogcount)

In [4]:
if REBUILD_DATA:
    dogvcats = DogsVSCats()
    dogvcats.make_training_data()
    

kagglecatsanddogs_3367a/PetImages/Cat


100%|████████████████████████████████████████████████████████████████████████████| 12501/12501 [02:53<00:00, 71.95it/s]


kagglecatsanddogs_3367a/PetImages/Dog


100%|████████████████████████████████████████████████████████████████████████████| 12501/12501 [03:17<00:00, 63.17it/s]


Cats: 12476
Dogs: 12470


In [6]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))


24946


In [15]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)
        
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        
        self.fc1 = nn.Linear(self._to_linear,512)
        self.fc2 = nn.Linear(512,2)
        
    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        
        print(x[0].shape)
        
        if self._to_linear == None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        
        return x
    
    
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1,self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x,dim=1)

In [16]:
net = Net()

torch.Size([128, 2, 2])


In [17]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(),lr=0.001)
loss_function = nn.MSELoss()

X=torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X= X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)



2494


In [18]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]


In [21]:
BATCH_SIZE = 100
EPOCHS =1


for epoch in range(EPOCHS):
    for i in tqdm(range(0,len(train_X),BATCH_SIZE)):
        batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
        batch_y = train_y[i:i+BATCH_SIZE]
        
        net.zero_grad()
        
        outputs = net(batch_X)
        loss = loss_function(outputs,batch_y)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch: {epoch}. Loss: {loss}")
    


  0%|                                                                                          | 0/225 [00:00<?, ?it/s]

torch.Size([128, 2, 2])



  0%|▎                                                                                 | 1/225 [00:00<03:22,  1.10it/s]

torch.Size([128, 2, 2])



  1%|▋                                                                                 | 2/225 [00:01<03:16,  1.14it/s]

...

torch.Size([128, 2, 2])



100%|███████████████████████████████████████████████████████████████████████████████▋| 224/225 [03:06<00:00,  1.14it/s]

torch.Size([128, 2, 2])



100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [03:06<00:00,  1.33it/s]

Epoch: 0. Loss: 0.2365850955247879


In [22]:
correct = 0
total =0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1,1,50,50))[0]
        pred_class = torch.argmax(net_out)
        
        if pred_class == real_class:
            correct +=1
        total+=1

print("Accuracy: ", round(correct/total, 3))
    


  0%|                                                                                         | 0/2494 [00:00<?, ?it/s]

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])



  0%|▎                                                                                | 8/2494 [00:00<00:32, 77.13it/s]

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])



  1%|▌                                                                               | 16/2494 [00:00<00:32, 76.57it/s]

...

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])



100%|█████████████████████████████████████████████████████████████████████████████▊| 2490/2494 [00:24<00:00, 99.99it/s]

torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])



100%|██████████████████████████████████████████████████████████████████████████████| 2494/2494 [00:24<00:00, 99.80it/s]

Accuracy:  0.625
